# Apps Architecture

* Tradeoff Agency-Reliability
* Decisions
    * Output
    * Next Step
    * Steps Availables

## LLM Cognitive Architectures

* Code
* LLM Call
* Chain
* Router
* State Machine
* Autonomous

### LLM Call

In [2]:
from typing import Annotated, TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama


model = ChatOllama(model="qwen:0.5b")

class State(TypedDict):
    # Messages have the type "list". The `add_messages` 
    # function in the annotation defines how this state should 
    # be updated (in this case, it appends new messages to the 
    # list, rather than replacing the previous messages)
    messages: Annotated[list, add_messages]

def chatbot(state: State):
    answer = model.invoke(state["messages"])
    return {"messages": [answer]}

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

graph = builder.compile()

In [4]:
graph.get_graph().draw_mermaid()

'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\tchatbot(chatbot)\n\t__end__([<p>__end__</p>]):::last\n\t__start__ --> chatbot;\n\tchatbot --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'

In [5]:
from langchain_core.messages import HumanMessage

input = {"messages": [HumanMessage('hi!')]}
for chunk in graph.stream(input):
    print(chunk)

{'chatbot': {'messages': [AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T18:46:20.6495927Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1804385800, 'load_duration': 1505442500, 'prompt_eval_count': 10, 'prompt_eval_duration': 87106800, 'eval_count': 10, 'eval_duration': 209551500, 'model_name': 'qwen:0.5b'}, id='run--7011b60f-825e-4c8c-89da-63c5776ef8d7-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20})]}}


### Chain (Flow Engineering) with extra steps !

In [254]:
from typing import Annotated, TypedDict, List, Tuple

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langchain_community.tools import QuerySQLDatabaseTool
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama


# useful to generate SQL query
model_low_temp = ChatOllama(model="qwen:0.5b",temperature=0)
# useful to generate natural language outputs
model_high_temp = ChatOllama(model="qwen:0.5b",temperature=0.8)
# summarize  content from table
model_summarizer = ChatOllama(model="qwen:0.5b", temperature=0.8)


In [255]:
class State(TypedDict):
    # to track conversation history
    messages: Annotated[list, add_messages]
    # input
    user_query: str
    # output
    sql_query: str
    sql_explanation: str
    query_result: List[Tuple]
    result_summary: str
    

class Input(TypedDict):
    user_query: str

class Output(TypedDict):
    sql_query: str
    sql_explanation: str
    query_result: List[Tuple]
    result_summary: str

In [256]:
generate_prompt = SystemMessage(
    """Given an input question create a SQL query according to question. Only return SQL query nothing else"""
)

def generate_sql(state: State) -> State:
    user_message = HumanMessage(state["user_query"])
    messages = [generate_prompt, *state["messages"], user_message]
    res = model_low_temp.invoke(messages)
    return {
        "sql_query": res.content,
        # update conversation history
        "messages": [user_message, res],
    }


In [257]:
from langchain_core.messages import trim_messages

trimmer = trim_messages(
    max_tokens=2,
    strategy="last",
    token_counter=len, #Each message is a token
    include_system=True,
    allow_partial=False,
)

explain_prompt = SystemMessage(
    "You are an assitant, given an SQL code input return a NON EMPTY explanation for that code"
)

def explain_sql(state: State) -> State:
    messages = [
        explain_prompt,
        # contains user's query and SQL query from prev step
        *state["messages"],
    ]
    msgs = trimmer.invoke(messages)
    res = model_high_temp.invoke(msgs)
    return {
        "sql_explanation": res.content,
        # update conversation history
        "messages": res,
    }

In [258]:
summarize_prompt = SystemMessage(
    "Given a list of tuples from a database summarize its content"
)
def summarize_result(state: State) -> State:
    query = state["sql_query"]
    db = SQLDatabase.from_uri("sqlite:///../data/sample.db")
    execute_query = QuerySQLDatabaseTool(db=db)
    table = execute_query.invoke(input=query)
    res = model_summarizer.invoke([summarize_prompt, table])
    return {
        "query_result": table,
        "result_summary": res.content,
        # update conversation history
        "messages": res,
    }

In [263]:
builder = StateGraph(State, input_schema=Input, output_schema=Output)
builder.add_node("generate_sql", generate_sql)
builder.add_node("explain_sql", explain_sql)
builder.add_node("summarize_result", summarize_result)
builder.add_edge(START, "generate_sql")
builder.add_edge("generate_sql", "explain_sql")
builder.add_edge("generate_sql", "summarize_result")
builder.add_edge("explain_sql", END)
builder.add_edge("summarize_result", END)
checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [264]:
config = {
    "configurable": {
        "thread_id": 1,
    }
}

In [265]:
graph.invoke({
  "user_query": "List the users"
}, config=config)

{'sql_query': 'SELECT * FROM users;',
 'sql_explanation': '',
 'query_result': "[(1, 'acf', '123'), (2, 'ale', '456'), (3, 'abc', '789'), (4, 'zyx', '987')]",
 'result_summary': '```sql\nSELECT * FROM table_name;\n```\n\nThis SQL statement will select all the columns from the `table_name` column.'}

In [266]:
states = list(graph.get_state_history(config))

for state in states:
    print(state.next)
    print(state.config["configurable"]["checkpoint_id"])
    print()

()
1f073cb3-5661-64df-8002-bc64fa77f5d3

('explain_sql', 'summarize_result')
1f073cb3-49d5-6962-8001-be457012c449

('generate_sql',)
1f073cb3-45e1-6f28-8000-daedfbecb3ce

('__start__',)
1f073cb3-45df-6877-bfff-86436f19618f



### Routing

In [269]:
from typing import Annotated, Literal, TypedDict

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_ollama import ChatOllama, OllamaEmbeddings

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

In [371]:
embeddings = OllamaEmbeddings(model="qwen:0.5b")
# useful to generate SQL query
model_low_temp = ChatOllama(model="qwen:0.5b" ,temperature=0)
# useful to generate natural language outputs
model_high_temp = ChatOllama(model="qwen:0.5b",temperature=0.7)

In [272]:
class State(TypedDict):
    # to track conversation history
    messages: Annotated[list, add_messages]
    # input
    user_query: str
    # output
    domain: Literal["Alejandro Cespon", "Greek Philosophy"]
    documents: list[Document]
    answer: str

class Input(TypedDict):
    user_query: str

class Output(TypedDict):
    documents: list[Document]
    answer: str

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdfloader = PyPDFLoader('../data/test.pdf')
alejandro_doc = pdfloader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splitted_docs = splitter.split_documents(alejandro_doc)

alejandro_records_store = InMemoryVectorStore.from_documents(splitted_docs, embeddings)
alejandro_records_retriever = alejandro_records_store.as_retriever()

In [275]:
from langchain_community.document_loaders import TextLoader

raw_documents = TextLoader('../data/philotest.txt', encoding="utf-8").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
    chunk_overlap=200)
documents = text_splitter.split_documents(raw_documents)

philo_records_store = InMemoryVectorStore.from_documents(documents, embeddings)
philo_records_retriever = philo_records_store.as_retriever()

In [398]:
router_prompt = SystemMessage(
    """Output only one word (alejandro or philosophy)"""
)

In [399]:
def router_node(state: State) -> State:
    user_message = HumanMessage(state["user_query"])
    messages = [router_prompt, *state["messages"], user_message]
    res = model_low_temp.invoke(messages)
    return {
        "domain": res.content,
        # update conversation history
        "messages": [user_message, res],
    }

In [400]:
from langchain.utils.math import cosine_similarity

def pick_retriever(
    state: State,
) -> Literal["retrieve_alejandro", "retrieve_philosophy"]:
    domain_embedding = embeddings.embed_query(state['domain'])
    alejandro_embedding = embeddings.embed_query("Alejandro Cespón Ferriol, professor, doctoral research, developer")
    philosophy_embeddings = embeddings.embed_query("Greek Philosophy, Nature, Origin, Math, Platon, Socrates")
    alejandro_sim = cosine_similarity([domain_embedding],[alejandro_embedding])
    philosophy_sim = cosine_similarity([domain_embedding], [philosophy_embeddings])

    print("Alejandro Sim")
    print(alejandro_sim)
    print("Philo Sim")
    print(philosophy_sim)
    if alejandro_sim > philosophy_sim:
        state["domain"] = "alejandro"
        return "retrieve_alejandro"
    else:
        state["domain"] = "philosophy"
        return "retrieve_philosophy"

In [401]:
def retrieve_alejandro(state: State) -> State:
    documents = alejandro_records_retriever.invoke(state["user_query"])
    return {
        "documents": documents,
    }

In [402]:
def retrieve_philosophy(state: State) -> State:
    documents = philo_records_retriever.invoke(state["user_query"])
    return {
        "documents": documents,
    }

In [403]:
alejandro_prompt = SystemMessage(
    """You are a helpful chatbot who answers questions based on the 
        curriculum of Alejandro Cespon Ferriol."""
)

philo_prompt = SystemMessage(
    """You are a helpful chatbot who answers questions about Greek Philosophy."""
)

In [404]:
def generate_answer(state: State) -> State:
    if state["domain"] == "alejandro":
        prompt = alejandro_prompt
    else:
        prompt = philo_prompt
    docs = state["documents"]
    messages = [
        prompt,
        *state["messages"],
        HumanMessage(f"Documents: { docs }"),
    ]
    res = model_high_temp.invoke(messages)
    return {
        "answer": res.content,
        # update conversation history
        "messages": res,
    }

In [405]:
builder = StateGraph(State, input_schema=Input, output_schema=Output)
builder.add_node("router", router_node)
builder.add_node("retrieve_alejandro", retrieve_alejandro)
builder.add_node("retrieve_philosophy", retrieve_philosophy)
builder.add_node("generate_answer", generate_answer)
builder.add_edge(START, "router")
builder.add_conditional_edges("router", pick_retriever)
builder.add_edge("retrieve_alejandro", "generate_answer")
builder.add_edge("retrieve_philosophy", "generate_answer")
builder.add_edge("generate_answer", END)

graph = builder.compile()

In [406]:
input = {
    "user_query": "Who is doctoral researcher?"
}
for c in graph.stream(input):
    print(c)

Alejandro Sim
[[0.54729499]]
Philo Sim
[[0.50112488]]
{'router': {'domain': 'Doctoral researcher.', 'messages': [HumanMessage(content='Who is doctoral researcher?', additional_kwargs={}, response_metadata={}, id='913579fd-76df-41ad-b54e-6f5112333ecf'), AIMessage(content='Doctoral researcher.', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T21:15:05.851385Z', 'done': True, 'done_reason': 'stop', 'total_duration': 719962800, 'load_duration': 74659800, 'prompt_eval_count': 28, 'prompt_eval_duration': 285308600, 'eval_count': 5, 'eval_duration': 338067800, 'model_name': 'qwen:0.5b'}, id='run--1e10100c-54b8-4bf9-b31d-803219bd16e9-0', usage_metadata={'input_tokens': 28, 'output_tokens': 5, 'total_tokens': 33})]}}
{'retrieve_alejandro': {'documents': [Document(id='5f8bd043-f265-4ac8-9bb5-026479c65ddf', metadata={'producer': 'MiKTeX-dvipdfmx (20220710)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-10T17:14:12-04:00', 'source': '../data/t